<a href="https://colab.research.google.com/github/pathstream-curriculum/Statistics/blob/master/Rideshare_Project3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Rideshare Project Part 3
<img src="https://data.cityofchicago.org/api/assets/73F1665C-0FE6-4183-8AD1-E91DB8EFAFA4?7CB02402-8E06-48B0-8C9A-3890182D58C7.png" width=400 alt="Drawing" style="width: 200px;"/>

The city of Chicago has hired you as an analyst to dive into their recently published dataset containing detailed information about all rides taken with rideshare providers like Uber and Lyft in Chicago and surrounding areas from November 2018 through March 2019.

In this third and final part of the project you'll first work though this notebook and then proceed to a couple of Google Sheets labs. The reason for this reordering of operations is that this part of the project gets a little bit complex, and we think it's a bit easier to follow what's going on in Python. With the context from this Python notebook, you'll dive into constructing models for rideshare usage in Google Sheets.

You'll begin with the same dataset, comprising two weeks worth of data from Dec. 21, 2018 to Jan. 3, 2019. After that you'll work with a larger dataset of roughly 90k randomly sampled records. Both datasets have been downsampled by a factor of 500 to reduce the size, meaning that only one of every 500 records (selected randomly) from this time period in the original dataset is included here. 

*Note: Some columns of unnessary or redundant information have been removed from the original data set and columns for Year, Month, Weekday and Hour of Day have been added for convenience. The original published data was anonymized by rounding off dollar amounts and times of day. To make the data more realistic looking we have added random noise to the Fare, Tip, Latitude and Longitude columns.*

**The first few steps of this project (reading in the data, removing null values and investigating descriptive statistics) are the same as for parts 1 and 2 of this project.**

### Scenario for this section of the project:
> The city of Chicago now wants you create a model for predicting the number of rides per day and per hour. Your goal is to create a model that the city can use to estimate the economic impact of rideshare activities over time. Follow along in this notebook to create your model! 

## Objectives for this Python lab:

1. Read in the dataset and compute summary statistics.

2. Create a model of daily rideshare usage.

3. Create a model of hourly rideshare usage.

The code for all these objectives is already written for you. All you need to do is press "Shift+Enter" on your keboard after selecting each cell to run the code. <mark>Be sure to run all the cells in order because some of the steps need to happen in a sequence</mark>. At a few points we give suggestions for making simple modifications to the code. Feel free to experiment with more if you're feeling curious!

### Objective 1: Read in the dataset and compute summary statistics.

#### 1.1 Investigate the data source (if you haven't already).

As always, a great first step before you jump into your analysis is to check out the source of your data. You can find out more about this exciting dataset [here](https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p/data).

#### 1.2 Import all the relevant Python packages.

Run the next cell to import Python packages (there won't be any output from this cell so if it runs without complaining you have successfully imported all packages)

In [0]:
# Import the pandas library for reading and manipulating your data
# Anywhere you see "pd" in this notebook it's a reference to the pandas library
import pandas as pd
# Extra step to ensure that pandas plays nice with matplotlib
pd.plotting.register_matplotlib_converters()
# Import the numpy library for running calculations on your data
# Anywhere you see "np" in this notebook it's a reference to the numpy library
import numpy as np
# Import a statistics package from scipy
from scipy import stats
# Import some components of the matplotlib library for plotting your data
# Anywhere you see "plt" or "mpimg" in this notebook it's a reference to the "pyplot" and "image" packages from matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
# Import seaborn library for making your plots pretty!
# Anywhere you see "sns" in this notebook it's a reference to the seaborn library
import seaborn as sns
# Set some default plotting parameters using seaborn
sns.set()


#### 1.3 Read in the dataset.

With the code below, you will use the `pandas` code library (`pd` for short) to read a csv (comma separated value) file containing your data into an object called "`df`". In this case, `df` is short for dataframe, which is a type of object used in Python for storing rows and columns of data.

Run the following cell (select and press play or "Shift+Enter" on the keyboard) to complete this step. 



In [0]:
# Read the data into a dataframe called "df".
url = "https://raw.githubusercontent.com/pathstream-curriculum/Statistics/master/rideshare_holidays.csv"
df = pd.read_csv(url, parse_dates=['Trip Start Timestamp', 'Trip End Timestamp'])

#### 1.4 Inspect the dataset.

Run the following cell to look at the column names and first few rows of your data. Be sure to scroll to the right to see all the columns. 

You can explore the [city of Chicago website for this dataset](https://data.cityofchicago.org/Transportation/Transportation-Network-Providers-Trips/m6dm-c72p/data) further to learn more about each column.

In [0]:
# Print the column names and first five rows of the dataset contained in df
# Note: to look at more than 5 rows just enter a number in the parentheses e.g., "df.head(10)"
df.head()

#### 1.4 Explore column data types and the presence of null / missing values.

In the cells below, running `df.shape` prints out the shape of the dataset and `df.info()` prints out each column name along with the total number of non-null values in that column and its data type. Investigate the output and see what you find!

In [0]:
# Display the shape of this dataset
df.shape

In [0]:
# Print information about the total number of non-null values and data types in each column of your dataframe
df.info()

#### 1.5 Handle missing values

 If there are null/missing values in your data, you will sometimes want to eliminate those rows from the dataset, or change them to an acceptable value. In some cases, null values may be interesting to explore further. For this project, you can choose to simply leave the null values alone or remove records with null values. 

---

If you want to remove all records containing null values, make a note of why you've decided to do this and then run the next cell after removing the "#" from the beginning of the line that says `df.dropna(inplace=True)` to drop all rows containing null values. If you choose not to remove missing values you can skip the next cell. 

In [0]:
# Remove all rows that contain any null values
#df.dropna(inplace=True) 
# Print information about the total number of non-null values and data types in each column of your dataframe
#df.info()

#### 1.6 Investigate summary statistics.

Run the cell below to compute and display summary statistics for your dataset. The output will be a table containing the count, mean, standard deviation, min, max and 25%, 50% (median) and 75% (Q1-Q3) quartiles for all columns. 

In [0]:
# Print out summary statistics for all columns
df.describe()

### Objective 2: Create a model of daily rideshare usage

The dataset you're working with contains a sampling (one out of every 500) of rides over the 2018/2019 holiday season (Dec. 21, 2018 through Jan. 3, 2019). Your first step toward creating a model of rideshare usage over time will be to simply calculate the average number of rides per day over this holiday period.

#### 2.1 Calculate average rides per day

Run the cell below to calculate the average number of rides per day over the 2018/2019 holiday season and print out that average. 

In [0]:
# First add a "Date" column to your dataframe by extracting the date from Trip Start Timestamp
df['Date'] = df['Trip Start Timestamp'].dt.date
# Create a new dataframe by grouping rides by "Date" and counting the of rides on each Date
daily_counts = df.groupby('Date').size().reset_index(name='Daily_Count')
# Multiply daily counts by 500 to account for the fact that the data has been downsampled by this factor
daily_counts['Daily_Countx500'] = daily_counts['Daily_Count'] * 500
# Extract the total number of days in the dataset by counting the number of individual Dates
total_days = daily_counts['Date'].count()
# Compute the average number of rides per day
avg_rides_per_day = daily_counts['Daily_Countx500'].mean()
# Compute the standard deviation for rides per day
std_rides_per_day = daily_counts['Daily_Countx500'].std()

# Print out the new dataframe as well as average number of rides per day and SEM
print(f'Average number of rides per day over the holiday season: {avg_rides_per_day:.0f} +/- {std_rides_per_day:.0f}')
print(f'Average taken over {total_days:.0f} days')
# Print out the counts of actual number of rides taken each day
daily_counts

#### 2.2 Plot rides per day

Run the next cell to plot the average rides per day you calculated above over a histogram of actual rides per day over the holiday period.

In [0]:
# Set how big to plot the figure below this cell showing rides per day.
plt.figure(figsize=(10,6))

# plot a histogram of rides per day
plt.bar(daily_counts['Date'], daily_counts['Daily_Countx500'], label='Actual Rides/Day')
plt.xticks(rotation=65)  # Rotate the x tick labels for readability

# Add labels to the x and y axes
plt.ylabel('Rides per Day', fontsize=20)
plt.xlabel('Date', fontsize=20)

# Plot the average as a horizontal red line
plt.axhline(y=avg_rides_per_day, xmin=0, xmax=1, c='r', label='Average Rides/Day +/- St.Dev.')
# Overlay the holiday dataset average +/- standard deviation as red shaded area
plt.fill_between(daily_counts['Date'], avg_rides_per_day-std_rides_per_day, 
                 avg_rides_per_day+std_rides_per_day, color='r', alpha=0.3, zorder=2)
plt.legend(fontsize=15)
plt.show()

# Print out the average number of rides per day and standard deviation
print(f'Average number of rides per day over the holiday season: {avg_rides_per_day:.0f} +/- {std_rides_per_day:.0f}')
print(f'Average taken over {total_days:.0f} days')

Congratulations! You just made a model of rideshare usage per day! The solid red line in the plot above represents the average number of rides per day over the holiday period and the dashed red lines represent the mean plus or minus the standard deviation. 

### Question:

1. What might you do to improve your model?

#### 2.3 Gather more data
One of the first things you might be curious about at this point is whether the holiday data is really representative of rideshare useage over different or longer time periods. More specifically, would rideshare usage look more consistent day to day in a different sample? And, is this average number of rides actually representative of the larger poulation? Indeed, one of the most common approaches to improving or better understanding your analytical results is to gather more data! 

In this case, you decide to gather a sampling of the full range of data from Nov. 1, 2018 to Mar. 31, 2019 again extracting just one out of every 500 records in the database to keep the data size manageable. This results in a new data set of around 90k records.

Run the next cell to read in a larger 90k sample spanning November to March.

In [0]:
# Read in a new dataset of ~90,000 randomly selected records
url = "https://raw.githubusercontent.com/pathstream-curriculum/Statistics/master/rideshare_random90k.csv"
df_90k = pd.read_csv(url, parse_dates=['Trip Start Timestamp', 'Trip End Timestamp'])

Run the next cell to have a look at the summary stats for this new data set

In [0]:
# Print out summary statistics for all columns
df_90k.describe()

#### 2.4 Calculate a new model based on the larger dataset

The first thing you'll do now is to compute the same model you did above for the holiday dataset, but now for the larger 90k sample. 

Run the cell below to calculate average rides per day for the 90k sample 

In [0]:
# First add a "Date" column to your dataframe by extracting the date from Trip Start Timestamp
df_90k['Date'] = df_90k['Trip Start Timestamp'].dt.floor('d')
# Create a new dataframe containing counts of rides by day
daily_counts_90k = df_90k.groupby('Date').size().reset_index(name='Daily_Count')
# Multiply daily counts by 500 to account for the fact that the data has been downsampled by this factor
daily_counts_90k['Daily_Countx500'] = daily_counts_90k['Daily_Count'] * 500
# Extract the total number of days in the dataset
total_days_90k = daily_counts_90k['Date'].count()
# Compute the average number of rides per day
avg_rides_per_day_90k = daily_counts_90k['Daily_Countx500'].mean()
# Compute the standard deviation of rides per day
std_rides_per_day_90k = daily_counts_90k['Daily_Countx500'].std()

# Print out the new dataframe as well as average number of rides per day and SEM
print(f'Number of rides per day: {avg_rides_per_day_90k:.0f} +/- {std_rides_per_day_90k:.0f}')
print(f'Average taken over {total_days_90k:.0f} days')
# Print out the first few rows of a dataframe containing actual daily ride counts
daily_counts_90k.head(10)

**Note: The printout above is just the first few rows of a much larger table including all dates from November through March.**

### Questions:
1. What do you notice about the number of rides per day in the full data set compared to what you found for the holiday data?

#### 2.5 Plot the new dataset and compare your two calculations of average rides per day
Run the cell below to plot and print a comparison with your results from the holiday data.

In [0]:
# Set how big to plot the figure below this cell showing rides per day.
plt.figure(figsize=(18,6))

# Plot rides per day for the full sample
plt.bar(daily_counts_90k['Date'], daily_counts_90k['Daily_Countx500'], label='Rides/Day Full Sample')
plt.xticks(rotation=65)  # Rotate the x tick labels for readability

# Overplot rides per day for the holiday sample in red
plt.bar(daily_counts['Date'], daily_counts['Daily_Countx500'], color='magenta', label='Rides/Day Holidays')
plt.xticks(rotation=65)  # Rotate the x tick labels for readability

# Add labels to the x and y axes
plt.ylabel('Rides per Day', fontsize=20)
plt.xlabel('Date', fontsize=20)

# Plot the holidays and full date range averages as a horizontal red and green lines
plt.axhline(y=avg_rides_per_day_90k, xmin=0, xmax=1, c='green', label='Avg. Rides/Day Full Date Range +/- St.Dev.')
plt.axhline(y=avg_rides_per_day, xmin=0, xmax=1, c='red', label='Avg. Rides/Day Holidays +/- St.Dev.')
# Overlay the full dataset average +/- standard deviation as green shaded area
plt.fill_between(daily_counts_90k['Date'], avg_rides_per_day_90k-std_rides_per_day_90k, 
                 avg_rides_per_day_90k+std_rides_per_day_90k, color='g', alpha=0.3, zorder=2)
# Overlay the holiday dataset average +/- standard deviation as red shaded area
plt.fill_between(daily_counts['Date'], avg_rides_per_day-std_rides_per_day, 
                 avg_rides_per_day+std_rides_per_day, color='r', alpha=0.6, zorder=2)

plt.legend(fontsize=15)
plt.show()

# Print out the average number of rides per day and standard deviation
print(f'Average number of rides per day over the full date range: {avg_rides_per_day_90k:.0f} +/- {std_rides_per_day_90k:.0f}')
print(f'Average taken over {total_days_90k:.0f} days')

### Questions:
1. How does the average number of rides per day for the holiday period compare with the larger November through March dataset? 
2. What else do you notice about the five month data set when it's plotted as a histogram above? How many peaks do you count in the histogram? How many weekends are there betwen November 1, 2018 and March 31, 2019?
3. What steps would you suggest to create a model that does a better job of predicting the number of rides there will actually be on any given day?



#### 2.6 Calculate the average number of rides for each weekday.

***Recognizing periodicity:***

The peaks in the histogram of rides per day above represent the number of rides taken on the weekends. This regular periodic signal demonstrates that the demand for rides on weekends is consistently higher than the demand during the middle of the week. Now your job is to use that information to improve your model! 

Your original model (the flat line average shown above) was a simple average over all time. The average of a periodic signal (like the demand for rides as a function of day of the week) is a reasonable measurement to make when you are concerned with averaging over timespans much longer than the period of the signal. For example, if you just wanted to know the average number of rides per year, you wouldn't be bothered by the fact that there are more rides on weekends than weekdays. 

***A switch from predicting average rides over all time to rides for an individual day of the week***

In many cases, as an analyst you'll be calculating an average to be able to predict some future measure of the metric you're interested in. For example, if you're interested in knowing how many rides are likely to happen next year, you might take an average over the last several years as a way of predicting that value.

If instead you're interested in predicting the number of rides that will be taken next Monday, then your best bet would be to average a bunch of past Mondays to get your answer, because, as you found in the plot above, the number of rides per day appears to depend strongly on which day of the week it is!



***Compute your new model***

In the next code cell, you'll compute the average number of rides per day for each day of the week and the standard deviation associated with each average.

Prepare yourself, the next cell contains some mind-bending Python code. There's a lot going on here but the important thing to understand is that you're calculating the average number of rides for each weekday and the standard deviation associated with that average.

In [0]:
# Start by creating a list of the days of the week
weekdays = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
# Add a column called "Weekday" to your daily_counts_90k dataframe
daily_counts_90k['Weekday'] = daily_counts_90k['Date'].apply(lambda x: weekdays[x.weekday()])
# Create a new dataframe with number of rides per weekday and number of occurences of each weekday in the 90k dataset
weekday_summary = pd.DataFrame(
    {
        'Weekday': weekdays,
        'Day_Count': [df_90k.loc[df_90k['Weekday']==day,'Trip Start Timestamp'].apply(lambda x: x.date()).nunique() for day in weekdays],
        'Avg_Ride_Count': daily_counts_90k.groupby('Weekday').mean().loc[weekdays,'Daily_Count'],
        'StDev_Ride_Count': daily_counts_90k.groupby('Weekday').std().loc[weekdays,'Daily_Count'],
        'Avg_Ride_Countx500': daily_counts_90k.groupby('Weekday').mean().loc[weekdays,'Daily_Countx500'],
        'StDev_Ride_Countx500': daily_counts_90k.groupby('Weekday').std().loc[weekdays,'Daily_Countx500']
    }
).set_index('Weekday')
# Print out the summary dataframe of average number of rides per weekday
weekday_summary

### Questions:
1. What was the average number of rides that occurred on a Monday in this data set? How about on a Thursday?
2. How many unique Tuesdays were there in the data set? How many Fridays?

#### 2.7 Plot your new model to predict counts and confidence for each day of the week
Now that you have recognized that the demand for rides fluctuates periodically on a weekly basis, and you have computed the actual demand for rides (total for each day) and how much it changes from week to week (the standard deviation) you're ready to create a new model! 

Run the next cell to convert your counts of rides and days into an estimate of rides per day and error.

Run the next cell to plot your new model against the data.

In [0]:

# Extract a multi-week model from the ride_counts dataframe (starting on a Thursday)
one_week_model = list(weekday_summary['Avg_Ride_Countx500'].values)
weekday_rides_model = np.array(one_week_model[3:] + 21*one_week_model)
one_week_error = list(weekday_summary['StDev_Ride_Countx500'].values)
weekday_rides_error = np.array(one_week_error[3:] + 21*one_week_error)
# Create a figure to display the model against actual rides per day
plt.figure(figsize=(18, 6))

# Plot daily ride counts for the 90k sample
plt.bar(daily_counts_90k['Date'], daily_counts_90k['Daily_Countx500'], label='Actual Rides/Day')

# Plot the full date range average as a horizontal green line
plt.axhline(y=avg_rides_per_day_90k, xmin=0, xmax=1, c='g', label='Avg. Rides/Day Full Date Range +/- St.Dev.')
# Overlay the full dataset average +/- standard deviation as green shaded area
plt.fill_between(daily_counts_90k['Date'], avg_rides_per_day_90k-std_rides_per_day_90k, 
                 avg_rides_per_day_90k+std_rides_per_day_90k, color='g', alpha=0.3, zorder=2)
# Plot the new weekday rides model in red
plt.plot(daily_counts_90k['Date'], weekday_rides_model, color='red', linewidth=2, label='Simple Model of Rides / Day +/- Std. Dev.')
plt.fill_between(daily_counts_90k['Date'], weekday_rides_model-weekday_rides_error, weekday_rides_model+weekday_rides_error, color='red', alpha=0.3, zorder=2)
plt.xticks(rotation=65)  # Rotate the x tick labels for readability
plt.ylabel('Rides per Day', fontsize=20)
plt.xlabel('Date', fontsize=20)
# Set the limits of the x-axis to any date range (default is the full range)
plt.xlim(pd.Timestamp('2018-11-1'), pd.Timestamp('2019-3-31'))
plt.legend(fontsize=15)
plt.show()

**Congratulations!** You have significantly improved your model! In the plot above, your average rides per day value for each day of the week is shown in red with a margin of error of +/- one standard deviation.  

The blue bars represent actual demand for rides over each day in this time period and everywhere that your model in red is overlapping with the tops of the blue bars, you are predicting the correct number of rides (to within your one standard deviation confidence interval). Just by calculating this simple average demand for rides over each day of the week you have created a powerful predictive model that captures the dynamic nature of your data!

#### 2.8 Investigate anomalies
Also immediately obvious in the plot above are some areas where the actual demand does not match the model. As an analyst, you are equally excited by the places where your model fails as where it succeeds! 

<img src="https://pathstream-data-analytics.s3-us-west-2.amazonaws.com/rides_per_day_model.png" width=600 alt="Drawing" style="width: 200px;"/>

You can see clearly that the holiday period (Dec. 21, 2018 - Jan. 3, 2019), which you investigated previously, has a lower overall demand for rides than any other time period in the data set. But what about those other departures from the model? 

To zoom in on specfic parts of the plot above, and see clearly which days represent discrepancies with your model, re-run the block of code above after changing the date range where you find this line of code:
```python
# Set the limits of the x-axis to any date range (default is the full range)
plt.xlim(pd.Timestamp('2018-11-1'), pd.Timestamp('2019-3-31'))
```
The defaults of Nov. 1 and Mar. 31 represent the first and last dates in the data set but you can change these to any other dates you like to zoom in on a section of the plot. Change these dates and investigate the following questions:

1. What would you expect is the reason for the low demand in late November?
2. What about that spike in mid-March? What might you guess is driving higher demand there?
3. Strangest of all, what is that one-day dip in late January?
4. What would you suggest as improvements you could make to your model of rides per day to improve its accuracy to better accommodate these days or weeks when the average model doesn't fit?

### Objective 3: Create an hourly model of rideshare usage.

#### 3.1 Extract a subset of data where your weekday model fits well.
You have done a fine job so far of creating a model that predicts the number of rides per day for each day of the week. Now the city wants you to improve the time resolution of your model to predict rides per hour throughout each day as well!

<img src="https://pathstream-data-analytics.s3-us-west-2.amazonaws.com/early_dec_rides.png" width=600 alt="Drawing" style="width: 200px;"/>


There are two weeks in December where your model appears to be a particlarly good fit (Dec. 3 - 16 shown above), so you select those dates to take a closer look. This data set spans two weeks from a Monday to a Sunday. Your next step is to extract this subset of data to investigate how your model fits when you increase the time resolution to hourly.

Run the next cell to extract the two week data set from early Dec., 2018.

Note: Running the next cell does not generate any output.


In [0]:
# Set the start date/time as the first timestamp in the dataframe
start_timestamp = pd.Timestamp(2018, 12, 3, 0)
# Set the end date/time as the last timestamp in the dataframe
end_timestamp = pd.Timestamp(2018, 12, 17, 0)
# Extract a new dataframe that only includes two weeks of data in early February
df_dec = df_90k[df_90k['Trip Start Timestamp'] > start_timestamp] 
df_dec = df_dec[df_dec['Trip Start Timestamp'] < end_timestamp]

#### 3.2 Extract hourly ride counts from your subset of data
Run the next cell to count rides by the hour over the two week sample you extracted above.

In [0]:
# Create a separate dataframe of rides per hour
hourly_counts = df_dec.groupby(['Date', 'Hour of Day']).size().reset_index(name='Hourly_Count')
# Multiply by downsampling factor to get to actual rides per hour
hourly_counts['Hourly_Countx500'] = hourly_counts['Hourly_Count']*500
# Create a column of timestamps centered on each hourly bin used for counting
add_interval = np.timedelta64(30, 'm')
hourly_counts['Bin_Center_Timestamp'] = df_dec['Trip Start Timestamp'].apply(lambda x: np.datetime64(x.strftime('%Y-%m-%d %H'))+add_interval).sort_values().unique()
hourly_counts.head(10)

#### 3.3 Create a new model by converting your daily model to its hourly equivalent

Run the next cell to take your daily rides model and make an hourly prediction by simply dividing daily rides by 24 (assuming an equal number of rides for each hour of the day)

Note: Running the next cell does not generate any output.

In [0]:
# Create lists to model values for every hour model and error
# Loop over the values for daily rides and error for each day of the week
# Note: the *24 at the end actually replicates each value 24 times a list rather than multiplying the value inside the brackets by 24
oneweek_hourly_model = [[daily_rides/24] * 24 for daily_rides in weekday_summary['Avg_Ride_Countx500'].values]
oneweek_hourly_error = [[daily_err/24] * 24 for daily_err in weekday_summary['StDev_Ride_Countx500'].values]

# Take the one week model and replicate it twice (list*2) and convert to a numerical array
avg_hourly_model = np.array(oneweek_hourly_model*2).ravel()
avg_hourly_error = np.array(oneweek_hourly_error*2).ravel()

#### 3.4 Plot your new hourly model against hourly ride counts from the two week subsample
Great! Now you have converted your daily rides model into its hourly equivalent. 

Run the next cell to plot actual counts per hour vs. your new model.

In [0]:
# Plot a histogram from start to end date binned by hour
plt.figure(figsize=(18,6))
# Plot a bar chart (histogram) of rides per hour
plt.bar(hourly_counts['Bin_Center_Timestamp'], hourly_counts['Hourly_Countx500'], width=0.05, label='Actual Rides/Hour')
# Overplot the average rides per hour (dividing rides per day by 24)
plt.axhline(y=avg_rides_per_day_90k/24, xmin=0, xmax=1, c='g', linewidth=1.5, label='Average Rides Per Hour Nov-Mar +/- SEM')
# Overlay the full dataset average +/- standard deviation as green shaded area
plt.fill_between(daily_counts_90k['Date'], (avg_rides_per_day_90k-std_rides_per_day_90k)/24, 
                 (avg_rides_per_day_90k+std_rides_per_day_90k)/24, color='g', alpha=0.3, zorder=2)
# Overplot the your new avg_hourly_model
plt.plot(hourly_counts['Bin_Center_Timestamp'], avg_hourly_model, color='darkorange', linewidth=3, label='Daily Model Converted to Hourly +/- Std. Dev.')
plt.fill_between(hourly_counts['Bin_Center_Timestamp'], avg_hourly_model-avg_hourly_error, 
                 avg_hourly_model+avg_hourly_error, color='darkorange', alpha=0.3, zorder=2)
plt.xticks(rotation=65) # Rotate the x tick labels for readability
plt.ylabel('Rides Per Hour', fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.legend(fontsize=15)
# Set the limits of the x-axis to any date range (default is the full range)
# The format for generating these timestamps is pd.Timestamp(YYYY, MM, DD, hour)
plt.xlim(pd.Timestamp(2018, 12, 3, 0), pd.Timestamp(2018, 12, 17, 0))
plt.show()

### Questions:
1. What do you notice now about your model when compared to hourly ride counts? Does it still look like a good fit?
2. What sort of periodicities do you now observe in the data?

### Zoom in to get the details
Just like you did before, you can take a closer look by zooming in on the data. To do this, change the limits of the x-axis by changing the timestamps in the following line of code:
```python
# Set the limits of the x-axis to any date range (default is the full range)
# The format for generating these timestamps is pd.Timestamp(YYYY, MM, DD, hour)
plt.xlim(pd.Timestamp(2018, 12, 3, 0), pd.Timestamp(2018, 12, 17, 0))
```
### Questions (after zooming in on a couple days):
1. How many peaks typically occur in any given day? What time of day do these peaks correspond to? Why do you think this occurs?
2. What would be your approach to improving your model?

#### 3.5 Make a better hourly model

In the plot above, you can see that simply dividing daily counts by 24 is not going to cut it as an hourly model. Again you see periodicity, so as the simplest possible approach you will now do the same thing you did to estimate the demand for rides as a function of day of the week, but this time for hour of the day. Also, you won't forget about what you learned from the daily rides analysis. You'll fold in the dependence on weekday into your hourly estimates as well!

Remember that, as you practiced in the lessons, if you want to use the mean and standard deviation to model your data, you'd like to investigate the distribution of the data to see if it looks roughly like a normal distribution. In principle, it would have been a good idea to try to determine whether weekday counts followed a normal distribution but with only a couple dozen examples of each weekday in the sample, your sample would be too small. 

With hour of day however, you have 151 days where you have a measurement for each hour of the day. With this sample you can start to get a handle on the distribution of counts at any given hour of the day across the sample. 

Run the next code cell to generate a pivot table of counts by hour of day for each date in your sample. 

In [0]:
# Create a pivot table of ride counts for each date by hour of day
hour_counts = pd.pivot_table(df_90k, index='Date', 
                                  columns='Hour of Day', 
                                  values='Trip Start Timestamp', 
                                  aggfunc=len)
# Print out the pivot table
hour_counts

The table above shows all the dates in your sample in the left-hand column, and the counts by hour of day across each row (with the hours listed across the top of the table). 

Run the next cell to look at the distribution for one particular hour of day compared to a normal distribution. 

In [0]:
# Choose an hour of the day.
hour = 11
# Extract counts for that hour of day from the pivot table.
counts = hour_counts[hour]
counts.dropna(inplace=True)
# Plot a histogram and fit the distribution with a normal curve.
sns.distplot(counts, fit=stats.norm, kde=False)
plt.xlabel(f'Ride Counts at Hour {hour}')

The plot you generated above shows the distribution of ride counts for a particular hour of day. You can change the `hour` variable to look at a different hour of day.

What you'll notice if you look at the ride counts for hours in the middle of the night when there are not as many riders, the distribution looks skewed. Unfortunately this is a downside of having downsampled the data by a factor of 500. 

If, for example, you find that the average number of rides at 3AM is only 5 rides, the actual number of rides is 500 times higher, or in other words, there's something like 2500 rides taken at 3AM in Chicago every day!

For this reason, it's best to make a comparison with the normal distribution where ride counts are higher in the midday hours. 

Questions:

1. Does it look like the ride counts follow a normal distribution for hours in the middle of the day?
2. Does it seem reasonable to model this distribution using the mean and standard deviation?



Given that the ride counts per hour look roughly normal, the next step is to proceed with creating a model for rides per hour that also takes into account the differences based on day of the week. 

Run the next two cells to compute your new hourly model by extracting counts per hour (in a slightly different format than you did above) over the entire 90k sample.

In [0]:
# Create a separate dataframe of rides per hour
hourly_counts_90k = df_90k.groupby(['Date', 'Hour of Day']).size().reset_index(name='Hourly_Count')
# Multiply by downsampling factor to get to actual rides per hour
hourly_counts_90k['Hourly_Countx500'] = hourly_counts_90k['Hourly_Count'] * 500
# Create a column of timestamps centered on each hourly bin used for counting
add_interval = np.timedelta64(30, 'm')
hourly_counts_90k['Bin_Center_Timestamp'] = df_90k['Trip Start Timestamp'].apply(lambda x: np.datetime64(x.strftime('%Y-%m-%d %H'))+add_interval).sort_values().unique()
hourly_counts_90k.head(10)

In [0]:
# Create a hourly rides summary by computing the average rides and standard deviation for each hour of the day
hourly_summary = pd.DataFrame({
    'unique_hour_counts': hourly_counts_90k.groupby('Hour of Day').agg('count')['Date'],
    'Avg_Ride_Count': hourly_counts_90k.groupby('Hour of Day').sum()['Hourly_Count']/total_days_90k,
    'StDev_Ride_Count': hourly_counts_90k.groupby('Hour of Day').std()['Hourly_Count'],
    'Avg_Ride_Countx500': hourly_counts_90k.groupby('Hour of Day').sum()['Hourly_Countx500']/total_days_90k,
    'StDev_Ride_Countx500': hourly_counts_90k.groupby('Hour of Day').std()['Hourly_Countx500']
})
# Print out your hourly summary
hourly_summary

#### 3.6 Plot your new model against two weeks of hourly counts data
Run the next cell to shape your hourly results into a model for plotting

Note: Running the next cell does not generate any output.

In [0]:
# Convert lists to numerical arrays and multiply by 500 (downsampling factor)
# Divide total hourly ride counts by the number of days in the 90k sample
hourly_rides = hourly_summary['Avg_Ride_Countx500']
hourly_rides_error = hourly_summary['StDev_Ride_Countx500']

# Loop through each day of the week creating a model by multiplying avg_hourly_rides and error by the conversion factor derived from the daily model
oneweek_hourly_model = np.array([hourly_rides*weekday_summary.loc[weekday, 'Avg_Ride_Countx500']/avg_rides_per_day_90k for weekday in weekdays]).ravel()
oneweek_hourly_error = np.array([hourly_rides_error*weekday_summary.loc[weekday, 'Avg_Ride_Countx500']/avg_rides_per_day_90k for weekday in weekdays]).ravel()

# Create a twoweek_hourly_model by replicating the array twice.
twoweek_hourly_model = np.tile(oneweek_hourly_model, 2)
twoweek_hourly_error = np.tile(oneweek_hourly_error, 2)

Run the next cell to plot up your new model for comparison!

In [0]:
# Plot a histogram from start to end date binned by hour
plt.figure(figsize=(18,8))
# Plot a bar chart (histogram) of rides per hour
plt.bar(hourly_counts['Bin_Center_Timestamp'], hourly_counts['Hourly_Countx500'], width=0.05)
# Overplot the average rides per hour (dividing rides per day by 24)
plt.axhline(y=avg_rides_per_day_90k/24, xmin=0, xmax=1, c='g', linewidth=1.5, label='Average Rides Per Hour Nov-Mar +/- St.Dev.')
# Overlay the full dataset average +/- standard deviation as green shaded area
plt.fill_between(daily_counts_90k['Date'], (avg_rides_per_day_90k-std_rides_per_day_90k)/24, 
                 (avg_rides_per_day_90k+std_rides_per_day_90k)/24, color='g', alpha=0.3, zorder=2)
# Overplot the your previous avg_hourly_model
plt.plot(hourly_counts['Bin_Center_Timestamp'], avg_hourly_model, color='darkorange', linewidth=3, label='Daily Model Converted to Hourly +/- Std. Dev.')
plt.fill_between(hourly_counts['Bin_Center_Timestamp'], avg_hourly_model-avg_hourly_error, avg_hourly_model+avg_hourly_error, color='darkorange', alpha=0.3, zorder=2)

# Overplot your new twoweek_hourly_model
plt.plot(hourly_counts['Bin_Center_Timestamp'], twoweek_hourly_model, color='purple', linewidth=3, label='New Hourly Model +/- Std. Dev.')
plt.fill_between(hourly_counts['Bin_Center_Timestamp'], 
                 twoweek_hourly_model-twoweek_hourly_error, 
                 twoweek_hourly_model+twoweek_hourly_error, 
                 color='purple', 
                 alpha=0.3, 
                 zorder=2)
plt.xticks(rotation=65) # Rotate the x tick labels for readability
plt.ylabel('Rides Per Hour', fontsize=20)
plt.xlabel('Date', fontsize=20)
plt.legend(loc='upper left', fontsize=15)
# Set the limits of the x-axis to any date range (default is the full range)
# The format for generating these timestamps is pd.Timestamp(YYYY, MM, DD, hour)
plt.xlim(pd.Timestamp(2018, 12, 3, 0), pd.Timestamp(2018, 12, 17, 0))
plt.show()

**Well done!** Very Impressive, you now have a model of the demand for rides at each hour of the day for each day of the week (in purple above)! As you can see, your model isn't perfect, but it does a better job of modeling hourly ride counts than either a flat average over many days (green) or a per-day average (orange). Now you are able to present the city of Chicago with a model that does a reasonable job of predicting the number of rides to expect on average for any day of the week and any hour of the day. You accomplished all of this without any fancy or complicated modeling, only simple averages and standard deviations. Nice work!

### Questions:
1. Where does your model seem to fit well? Where does it not fit so well?
2. At this point, what kind of approach might you take to improving your model?

To investigate your model fit more closely, you can again zoom in to view individual days by changing the limits on the x-axis where you find this line of code above:
```python
# Set the limits of the x-axis to any date range (default is the full range)
# The format for generating these timestamps is pd.Timestamp(YYYY, MM, DD, hour)
plt.xlim(pd.Timestamp(2018, 12, 3, 0), pd.Timestamp(2018, 12, 17, 0))
```

# Congratulations! You've come to the end of this notebook!

Great job building models of rideshare usage from the Chicago rideshare dataset. Now would be a good time to take any remaining notes or screenshots of anything you would like to be able to use and/or reference later.


Proceed to the next labs in Google Sheets to get a more hands-on experience building periodic models with this dataset. 